In [1]:
%load_ext autoreload
%autoreload 2

import sys
import math
import pickle
import random

import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pymesh

sys.path.append("../../")

import implicitmodules.torch as dm

In [2]:
seed = 1337
refinement_order = 2
sigma = 2.
sphere_count = 5
spheres = []

random.seed(seed)

for i in range(sphere_count):
    mesh = pymesh.generate_icosphere(0.5, [0., 0., 0.], refinement_order)

    axis = random.randint(0, 2)
    scale_matrix = torch.eye(3)
    scale_matrix[axis, axis] = random.gauss(0., sigma)

    deformed_vertices = dm.Utilities.linear_transform(torch.tensor(mesh.vertices, dtype=torch.get_default_dtype()), scale_matrix)
    spheres.append(dm.Models.DeformableMesh(deformed_vertices, torch.tensor(mesh.faces, dtype=torch.long)))

template_mesh = pymesh.generate_icosphere(0.5, [0., 0., 0.], refinement_order)
template = dm.Models.DeformableMesh(torch.tensor(template_mesh.vertices, dtype=torch.get_default_dtype()), torch.tensor(template_mesh.faces, dtype=torch.long))

print("Vertices count={vertices_count}".format(vertices_count=template_mesh.vertices.shape[0]))

AttributeError: module 'pymesh' has no attribute 'generate_icosphere'

In [3]:
pwd

'/home/gris/projects_py3/implicitmodules/script/torch'

In [2]:
with open("../../data/spheres.pickle", 'rb') as f:
    spheres = pickle.load(f)

AttributeError: Can't get attribute 'DeformableMesh' on <module 'implicitmodules.torch.Models.deformable' from '../../implicitmodules/torch/Models/deformable.py'>

In [5]:
%matplotlib qt5
side_count = math.ceil(math.sqrt(sphere_count))
for i, sphere in enumerate(spheres):
    ax = plt.subplot(side_count, side_count, i+1, projection='3d')
    points, triangles = sphere.geometry
    ax.plot_trisurf(points[:, 0].numpy(), points[:, 1].numpy(), points[:, 2].numpy(), triangles=triangles)
    dm.Utilities.set_aspect_equal_3d(ax)

plt.show()

In [3]:
sigma = 0.15
aabb = dm.Utilities.AABB.build_from_points(torch.tensor(template_mesh.vertices))

implicit1_points = aabb.fill_uniform_density(50)
implicit1_rot = torch.eye(3).repeat(implicit1_points.shape[0], 1, 1)

# implicit1_c = 0.2*torch.randn(implicit1_points.shape[0], 3, 1) + 1.
# implicit1_c.requires_grad_()

implicit1_c = torch.zeros(implicit1_points.shape[0], 3, 3)
implicit1_c[:, 0, 0] = 1.
implicit1_c[:, 1, 1] = 1.
implicit1_c[:, 2, 2] = 1.

In [4]:
global_translation = dm.DeformationModules.GlobalTranslation(3)

implicit1 = dm.DeformationModules.ImplicitModule1(3, implicit1_points.shape[0], sigma, implicit1_c, nu=0.1, gd=(implicit1_points, implicit1_rot))

In [5]:
sigmas_varifold = [0.5]
attachment = dm.Attachment.VarifoldAttachment(3, sigmas_varifold)

atlas = dm.Models.AtlasModel(template, [global_translation, implicit1], [attachment], len(spheres), lam=100., optimise_template=False, ht_nu=0.1, ht_sigma=0.2, fit_gd=None)

In [ ]:
shoot_solver = 'euler'
shoot_it = 10

costs = {}
fitter = dm.Models.Fitter(atlas, optimizer='torch_lbfgs')

fitter.fit(spheres, 50, costs=costs, options={'shoot_solver': shoot_solver, 'shoot_it': shoot_it, 'line_search_fn': 'strong_wolfe'})

In [7]:
costs = {}
fitter = dm.Models.Fitter(atlas, optimizer='torch_lbfgs')

fitter.fit(spheres, 50, costs=costs, options={'shoot_solver': shoot_solver, 'shoot_it': shoot_it, 'line_search_fn': 'strong_wolfe'})

Starting optimization with method torch LBFGS
Initial cost={'deformation': 10.825325012207031, 'attach': 277.0068054199219}
Time: 312.5562305810163
Iteration: 0
Costs
deformation=10.683792114257812
attach=276.9771423339844
Total cost=287.6609344482422
Time: 624.5318335850025
Iteration: 1
Costs
deformation=10.480719566345215
attach=276.9474792480469
Total cost=287.4281988143921
Time: 911.3658550209948
Iteration: 2
Costs
deformation=10.185215950012207
attach=276.76019287109375
Total cost=286.94540882110596
Time: 1210.2227594080032
Iteration: 3
Costs
deformation=9.760088920593262
attach=276.5892333984375
Total cost=286.34932231903076
Time: 1496.7144570479868
Iteration: 4
Costs
deformation=9.419245719909668
attach=276.0926818847656
Total cost=285.5119276046753
Time: 1798.2892822549911
Iteration: 5
Costs
deformation=9.380276679992676
attach=275.35699462890625
Total cost=284.7372713088989
Time: 2099.0464380710036
Iteration: 6
Costs
deformation=9.07970905303955
attach=274.8736267089844
Total 

In [ ]:
with torch.autograd.no_grad():
    template = atlas.compute_template()
    deformed_template = atlas.compute_deformed(shoot_solver, shoot_it)

In [ ]:
%matplotlib qt5
side_count = math.ceil(math.sqrt(sphere_count))
for i, sphere in enumerate(spheres):
    ax = plt.subplot(side_count, side_count, i+1, projection='3d')
    points, triangles = sphere.geometry
    ax.plot_trisurf(points[:, 0].numpy(), points[:, 1].numpy(), points[:, 2].numpy(), triangles=triangles, color='blue')
    dm.Utilities.set_aspect_equal_3d(ax)
    ax.plot_trisurf(template[:, 0].numpy(), template[:, 1].numpy(), template[:, 2].numpy(), triangles=triangles, color='red')

plt.show()